# handlers

> Segmentation workflow handlers — init, split, merge, undo, reset, AI split

In [ ]:
#| default_exp routes.handlers

In [ ]:
#| export
from typing import List, Dict, Any, Tuple, Callable, Optional, NamedTuple

from fasthtml.common import APIRouter, Script, Div

from cjm_fasthtml_card_stack.core.models import CardStackState
from cjm_fasthtml_card_stack.core.constants import DEFAULT_VISIBLE_COUNT, DEFAULT_CARD_WIDTH

from cjm_fasthtml_interactions.core.state_store import get_session_id
from cjm_workflow_state.history import pop_history

from cjm_transcript_segmentation.models import TextSegment, SegmentationUrls
from cjm_transcript_segmentation.components.step_renderer import (
    render_seg_column_body, render_seg_stats, render_toolbar,
    render_seg_source_position,
)
from cjm_transcript_segmentation.components.card_stack_config import (
    SEG_CS_IDS, SEG_TS_IDS,
)
from cjm_transcript_segmentation.utils import (
    word_index_to_char_position,
)
from cjm_transcript_segmentation.services.segmentation import (
    SegmentationService, split_segment_at_position, merge_text_segments,
    reindex_segments, reconstruct_source_blocks,
)
from cjm_transcript_segmentation.routes.core import (
    WorkflowStateStore, DEFAULT_MAX_HISTORY_DEPTH,
    _to_segments, _load_seg_context, _get_seg_state,
    _get_selection_state, _update_seg_state, _push_history,
    _build_card_stack_state,
)
from cjm_transcript_segmentation.routes.card_stack import (
    _build_slots_oob, _build_nav_response
)
from cjm_transcript_source_select.services.source import SourceService
from cjm_transcript_segmentation.html_ids import SegmentationHtmlIds

# Debug flag for segmentation handler tracing (set False in production)
DEBUG_SEG_HANDLERS = True

## Mutation Response Builder

Assembles the full OOB response for handlers that mutate segment data.
Includes decomposition-specific elements (stats, toolbar) in addition to card stack elements.

In [ ]:
#| export
def _build_mutation_response(
    segment_dicts:List[Dict[str, Any]],  # Serialized segments
    focused_index:int,  # Currently focused segment index
    visible_count:int,  # Number of visible cards
    history_depth:int,  # Current undo history depth
    urls:SegmentationUrls,  # URL bundle
    is_split_mode:bool=False,  # Whether split mode is active
    is_auto_mode:bool=False,  # Whether card count is in auto-adjust mode
) -> Tuple:  # OOB elements (slots + progress + focus + stats + toolbar + source position)
    """Build the standard OOB response for mutation handlers.
    
    Returns domain-specific OOB elements. The combined layer wrapper
    adds cross-domain elements (mini-stats badge, alignment status).
    """
    state = CardStackState(
        focused_index=focused_index,
        visible_count=visible_count,
        active_mode="split" if is_split_mode else None,
    )

    # Library handles: slots + progress + focus
    nav_response = _build_nav_response(segment_dicts, state, urls)

    # Segmentation-specific OOB elements
    segments = _to_segments(segment_dicts)
    stats_oob = render_seg_stats(segments, oob=True)
    toolbar_oob = render_toolbar(
        reset_url=urls.reset, ai_split_url=urls.ai_split, undo_url=urls.undo,
        can_undo=(history_depth > 0), visible_count=visible_count,
        is_auto_mode=is_auto_mode, oob=True,
    )
    source_pos_oob = render_seg_source_position(segments, focused_index, oob=True)

    return (*nav_response, stats_oob, toolbar_oob, source_pos_oob)

## Initialize Handler

In [ ]:
#| export
class SegInitResult(NamedTuple):
    """Result from pure segmentation init handler.
    
    Contains domain-specific data for the combined layer wrapper to use
    when building cross-domain OOB elements (KB system, shared chrome).
    """
    column_body: Any  # Rendered column body content
    segments: List[TextSegment]  # Initialized segments
    focused_index: int  # Focused segment index (always 0 on init)
    visible_count: int  # Visible card count
    card_width: int  # Card stack width in rem
    history_depth: int  # History depth (always 0 on init)
    is_auto_mode: bool  # Whether card count is in auto-adjust mode

In [ ]:
#| export
async def _handle_seg_init(
    state_store: WorkflowStateStore,  # The workflow state store
    workflow_id: str,  # The workflow identifier
    source_service: SourceService,  # Service for fetching source blocks
    segmentation_service: SegmentationService,  # Service for NLTK sentence splitting
    request,  # FastHTML request object
    sess,  # FastHTML session object
    urls: SegmentationUrls,  # URL bundle for segmentation routes
    visible_count: int = DEFAULT_VISIBLE_COUNT,  # Number of visible cards
    card_width: int = DEFAULT_CARD_WIDTH,  # Card stack width in rem
) -> SegInitResult:  # Pure domain result for wrapper to use
    """Initialize segments from Phase 1 selected sources.
    
    Returns pure domain data. The combined layer wrapper adds cross-domain
    coordination (KB system, shared chrome, alignment status).
    """
    if DEBUG_SEG_HANDLERS:
        print("[SEG_HANDLERS] _handle_seg_init called")

    session_id = get_session_id(sess)

    # Get selected sources from Phase 1
    selection_state = _get_selection_state(state_store, workflow_id, session_id)
    selected_sources = selection_state.get("selected_sources", [])

    # Read stored viewport preferences (may exist from previous session)
    seg_state = _get_seg_state(state_store, workflow_id, session_id)
    stored_visible_count = seg_state.get("visible_count", visible_count)
    stored_is_auto_mode = seg_state.get("is_auto_mode", False)
    stored_card_width = seg_state.get("card_width", card_width)

    if not selected_sources:
        # No sources selected, initialize with empty state
        _update_seg_state(
            state_store, workflow_id, session_id,
            segments=[], initial_segments=[],
            is_initialized=True, focused_index=0,
            history=[], visible_count=stored_visible_count,
            card_width=stored_card_width,
        )
        segments = []
    else:
        # Fetch source blocks via service API
        source_blocks = source_service.get_source_blocks(selected_sources)

        # Use segmentation service to split into sentences
        working_segments = await segmentation_service.split_combined_sources_async(
            source_blocks
        )
        segment_dicts = [s.to_dict() for s in working_segments]

        # Store in state
        _update_seg_state(
            state_store, workflow_id, session_id,
            segments=segment_dicts,
            initial_segments=segment_dicts.copy(),
            is_initialized=True, focused_index=0,
            history=[], visible_count=stored_visible_count,
            card_width=stored_card_width,
        )
        segments = _to_segments(segment_dicts)

    focused_index = 0
    history_depth = 0

    # Render column body (KB system managed by combined layer)
    column_body = render_seg_column_body(
        segments=segments,
        focused_index=focused_index,
        visible_count=stored_visible_count,
        card_width=stored_card_width,
        urls=urls,
        kb_system=None,
    )

    if DEBUG_SEG_HANDLERS:
        print("[SEG_HANDLERS] Returning SegInitResult")

    return SegInitResult(
        column_body=column_body,
        segments=segments,
        focused_index=focused_index,
        visible_count=stored_visible_count,
        card_width=stored_card_width,
        history_depth=history_depth,
        is_auto_mode=stored_is_auto_mode,
    )

## Split Handler

In [ ]:
#| export
async def _handle_seg_split(
    state_store: WorkflowStateStore,  # The workflow state store
    workflow_id: str,  # The workflow identifier
    request,  # FastHTML request object
    sess,  # FastHTML session object
    segment_index: int,  # Index of segment to split
    urls: SegmentationUrls,  # URL bundle for segmentation routes
    max_history_depth: int = DEFAULT_MAX_HISTORY_DEPTH,  # Maximum history stack depth
):  # OOB slot updates with stats, progress, focus, and toolbar
    """Split a segment at the specified word position."""
    session_id = get_session_id(sess)
    ctx = _load_seg_context(state_store, workflow_id, session_id)

    # Extract word index from token selector hidden input
    form = await request.form()
    word_index = int(form.get(SEG_TS_IDS.anchor_name, 0))

    # Validate index
    if segment_index < 0 or segment_index >= len(ctx.segment_dicts):
        state = _build_card_stack_state(ctx)
        return _build_slots_oob(ctx.segment_dicts, state, urls)

    # Push current state to history before modification
    history_depth = _push_history(
        state_store, workflow_id, session_id,
        ctx.segment_dicts, segment_index, max_history_depth,
    )

    # Get the segment and convert word index to character position
    segment = TextSegment.from_dict(ctx.segment_dicts[segment_index])
    char_position = word_index_to_char_position(segment.text, word_index)

    # Can't split at beginning or end
    if char_position <= 0 or char_position >= len(segment.text):
        return _build_mutation_response(
            ctx.segment_dicts, segment_index, ctx.visible_count, history_depth, urls,
            is_auto_mode=ctx.is_auto_mode,
        )

    # Split the segment
    first_seg, second_seg = split_segment_at_position(segment, char_position)

    # Build and reindex new segments list
    new_segments = ctx.segment_dicts[:segment_index]
    new_segments.append(first_seg.to_dict())
    new_segments.append(second_seg.to_dict())
    new_segments.extend(ctx.segment_dicts[segment_index + 1:])

    reindexed = reindex_segments(_to_segments(new_segments))
    new_segment_dicts = [s.to_dict() for s in reindexed]

    # Update state — focus moves to the new segment (second half)
    new_focused_index = segment_index + 1
    _update_seg_state(
        state_store, workflow_id, session_id,
        segments=new_segment_dicts, focused_index=new_focused_index,
    )

    return _build_mutation_response(
        new_segment_dicts, new_focused_index, ctx.visible_count, history_depth, urls,
        is_auto_mode=ctx.is_auto_mode,
    )

## Merge Handler

In [ ]:
#| export
def _build_merge_reject_flash(
    prev_index:int,  # Index of the segment above the boundary
    curr_index:int,  # Index of the segment below the boundary
) -> Div:  # OOB div containing JS that flashes both boundary cards
    """Build an OOB element that flashes both cards at a source boundary."""
    prev_id = SegmentationHtmlIds.segment_card(prev_index)
    curr_id = SegmentationHtmlIds.segment_card(curr_index)
    return Div(
        Script(f"""(function() {{
    var c1 = document.getElementById('{prev_id}');
    var c2 = document.getElementById('{curr_id}');
    [c1, c2].forEach(function(c) {{
        if (c) {{ c.classList.remove('bg-base-100'); c.classList.add('bg-error'); }}
    }});
    setTimeout(function() {{
        [c1, c2].forEach(function(c) {{
            if (c) {{ c.classList.remove('bg-error'); c.classList.add('bg-base-100'); }}
        }});
    }}, 400);
}})();"""),
        id=SegmentationHtmlIds.SCRIPT_RUNNER,
        hx_swap_oob="innerHTML",
    )

In [ ]:
#| export
def _handle_seg_merge(
    state_store: WorkflowStateStore,  # The workflow state store
    workflow_id: str,  # The workflow identifier
    request,  # FastHTML request object
    sess,  # FastHTML session object
    segment_index: int,  # Index of segment to merge (merges with previous)
    urls: SegmentationUrls,  # URL bundle for segmentation routes
    max_history_depth: int = DEFAULT_MAX_HISTORY_DEPTH,  # Maximum history stack depth
):  # OOB slot updates with stats, progress, focus, and toolbar
    """Merge a segment with the previous segment."""
    session_id = get_session_id(sess)
    ctx = _load_seg_context(state_store, workflow_id, session_id)
    
    # Can't merge first segment (nothing before it)
    if segment_index <= 0 or segment_index >= len(ctx.segment_dicts):
        state = _build_card_stack_state(ctx)
        return _build_slots_oob(ctx.segment_dicts, state, urls)
    
    # Check source boundary — reject merge across different audio sources
    prev_segment = TextSegment.from_dict(ctx.segment_dicts[segment_index - 1])
    curr_segment = TextSegment.from_dict(ctx.segment_dicts[segment_index])
    
    if (prev_segment.source_id is not None and
        curr_segment.source_id is not None and
        prev_segment.source_id != curr_segment.source_id):
        if DEBUG_SEG_HANDLERS:
            print(f"[SEG_HANDLERS] Merge rejected at source boundary: "
                  f"'{prev_segment.source_id}' != '{curr_segment.source_id}'")
        state = _build_card_stack_state(ctx)
        no_op = _build_slots_oob(ctx.segment_dicts, state, urls)
        flash = _build_merge_reject_flash(segment_index - 1, segment_index)
        return (*no_op, flash)
    
    # Push current state to history
    history_depth = _push_history(
        state_store, workflow_id, session_id,
        ctx.segment_dicts, segment_index, max_history_depth,
    )
    
    # Merge segments
    merged = merge_text_segments(prev_segment, curr_segment)
    
    # Build and reindex new segments list
    new_segments = ctx.segment_dicts[:segment_index - 1]
    new_segments.append(merged.to_dict())
    new_segments.extend(ctx.segment_dicts[segment_index + 1:])
    
    reindexed = reindex_segments(_to_segments(new_segments))
    new_segment_dicts = [s.to_dict() for s in reindexed]
    
    # Update state — focus moves to merged segment (previous position)
    new_focused_index = segment_index - 1
    _update_seg_state(
        state_store, workflow_id, session_id,
        segments=new_segment_dicts, focused_index=new_focused_index,
    )
    
    return _build_mutation_response(
        new_segment_dicts, new_focused_index, ctx.visible_count, history_depth, urls,
        is_auto_mode=ctx.is_auto_mode,
    )

## Undo Handler

In [ ]:
#| export
def _handle_seg_undo(
    state_store: WorkflowStateStore,  # The workflow state store
    workflow_id: str,  # The workflow identifier
    request,  # FastHTML request object
    sess,  # FastHTML session object
    urls: SegmentationUrls,  # URL bundle for segmentation routes
):  # OOB slot updates with stats, progress, focus, and toolbar
    """Undo the last operation by restoring previous state from history."""
    session_id = get_session_id(sess)
    ctx = _load_seg_context(state_store, workflow_id, session_id)
    
    result = pop_history(ctx.history)
    if result is None:
        state = _build_card_stack_state(ctx)
        return _build_slots_oob(ctx.segment_dicts, state, urls)
    
    snapshot, remaining_history = result
    previous_segments = snapshot["segments"]
    new_focused_index = min(snapshot["focused_index"], max(0, len(previous_segments) - 1))
    
    _update_seg_state(
        state_store, workflow_id, session_id,
        segments=previous_segments, history=remaining_history,
        focused_index=new_focused_index,
    )
    
    return _build_mutation_response(
        previous_segments, new_focused_index, ctx.visible_count, len(remaining_history), urls,
        is_auto_mode=ctx.is_auto_mode,
    )

## Reset Handler

In [ ]:
#| export
def _handle_seg_reset(
    state_store: WorkflowStateStore,  # The workflow state store
    workflow_id: str,  # The workflow identifier
    request,  # FastHTML request object
    sess,  # FastHTML session object
    urls: SegmentationUrls,  # URL bundle for segmentation routes
    max_history_depth: int = DEFAULT_MAX_HISTORY_DEPTH,  # Maximum history stack depth
):  # OOB slot updates with stats, progress, focus, and toolbar
    """Reset segments to the initial NLTK split result."""
    session_id = get_session_id(sess)
    ctx = _load_seg_context(state_store, workflow_id, session_id)
    seg_state = _get_seg_state(state_store, workflow_id, session_id)
    initial_segments = seg_state.get("initial_segments", [])
    
    # Push current state to history before reset
    history_depth = 0
    if ctx.segment_dicts:
        history_depth = _push_history(
            state_store, workflow_id, session_id,
            ctx.segment_dicts, ctx.focused_index, max_history_depth,
        )
    
    # Restore initial segments — reset focus to first segment
    _update_seg_state(
        state_store, workflow_id, session_id,
        segments=initial_segments.copy(), focused_index=0,
    )
    
    return _build_mutation_response(
        initial_segments, 0, ctx.visible_count, history_depth, urls,
        is_auto_mode=ctx.is_auto_mode,
    )

## AI Split Handler

In [ ]:
#| export
async def _handle_seg_ai_split(
    state_store: WorkflowStateStore,  # The workflow state store
    workflow_id: str,  # The workflow identifier
    segmentation_service: SegmentationService,  # Service for NLTK sentence splitting
    request,  # FastHTML request object
    sess,  # FastHTML session object
    urls: SegmentationUrls,  # URL bundle for segmentation routes
    max_history_depth: int = DEFAULT_MAX_HISTORY_DEPTH,  # Maximum history stack depth
):  # OOB slot updates with stats, progress, focus, and toolbar
    """Re-run AI (NLTK) sentence splitting on all current text."""
    session_id = get_session_id(sess)
    ctx = _load_seg_context(state_store, workflow_id, session_id)
    
    if not ctx.segment_dicts:
        state = _build_card_stack_state(ctx)
        return _build_slots_oob([], state, urls)
    
    # Push current state to history
    history_depth = _push_history(
        state_store, workflow_id, session_id,
        ctx.segment_dicts, ctx.focused_index, max_history_depth,
    )
    
    # Reconstruct source blocks from current segments
    source_blocks = reconstruct_source_blocks(ctx.segment_dicts)
    
    # Re-run NLTK splitting
    working_segments = await segmentation_service.split_combined_sources_async(
        source_blocks
    )
    new_segment_dicts = [s.to_dict() for s in working_segments]
    
    # Update state — reset focus to first segment
    _update_seg_state(
        state_store, workflow_id, session_id,
        segments=new_segment_dicts, focused_index=0,
    )
    
    return _build_mutation_response(
        new_segment_dicts, 0, ctx.visible_count, history_depth, urls,
        is_auto_mode=ctx.is_auto_mode,
    )

## Router Initialization

Creates the workflow router with init, split, merge, undo, reset, and AI split routes.

In [ ]:
#| export
def init_workflow_router(
    state_store: WorkflowStateStore,  # The workflow state store
    workflow_id: str,  # The workflow identifier
    source_service: SourceService,  # Service for fetching source blocks
    segmentation_service: SegmentationService,  # Service for NLTK sentence splitting
    prefix: str,  # Route prefix (e.g., "/workflow/seg/workflow")
    urls: SegmentationUrls,  # URL bundle (populated after routes defined)
    max_history_depth: int = DEFAULT_MAX_HISTORY_DEPTH,  # Maximum history stack depth
    handler_init: Callable = None,  # Optional wrapped init handler
    handler_split: Callable = None,  # Optional wrapped split handler
    handler_merge: Callable = None,  # Optional wrapped merge handler
    handler_undo: Callable = None,  # Optional wrapped undo handler
    handler_reset: Callable = None,  # Optional wrapped reset handler
    handler_ai_split: Callable = None,  # Optional wrapped ai_split handler
) -> Tuple[APIRouter, Dict[str, Callable]]:  # (router, route_dict)
    """Initialize workflow routes for segmentation.
    
    Accepts optional handler overrides for wrapping with cross-domain
    coordination (e.g., KB system, shared chrome, alignment status).
    """
    router = APIRouter(prefix=prefix)

    # Use provided handlers or fall back to raw domain handlers
    _init = handler_init or _handle_seg_init
    _split = handler_split or _handle_seg_split
    _merge = handler_merge or _handle_seg_merge
    _undo = handler_undo or _handle_seg_undo
    _reset = handler_reset or _handle_seg_reset
    _ai_split = handler_ai_split or _handle_seg_ai_split

    # -------------------------------------------------------------------------
    # Workflow Operations
    # -------------------------------------------------------------------------

    @router
    async def init(request, sess):
        """Initialize segments from Phase 1 selected sources."""
        return await _init(
            state_store, workflow_id, source_service, segmentation_service,
            request, sess, urls=urls,
        )

    @router
    async def split(request, sess, segment_index: int):
        """Split a segment at the specified word position."""
        return await _split(
            state_store, workflow_id, request, sess, segment_index,
            urls=urls, max_history_depth=max_history_depth,
        )

    @router
    async def merge(request, sess, segment_index: int):
        """Merge a segment with the previous segment."""
        result = _merge(
            state_store, workflow_id, request, sess, segment_index,
            urls=urls, max_history_depth=max_history_depth,
        )
        if hasattr(result, '__await__'):
            return await result
        return result

    @router
    async def undo(request, sess):
        """Undo the last segmentation operation."""
        result = _undo(state_store, workflow_id, request, sess, urls=urls)
        if hasattr(result, '__await__'):
            return await result
        return result

    @router
    async def reset(request, sess):
        """Reset segments to the initial NLTK split result."""
        result = _reset(
            state_store, workflow_id, request, sess,
            urls=urls, max_history_depth=max_history_depth,
        )
        if hasattr(result, '__await__'):
            return await result
        return result

    @router
    async def ai_split(request, sess):
        """Re-run AI (NLTK) sentence splitting on all current text."""
        return await _ai_split(
            state_store, workflow_id, segmentation_service, request, sess,
            urls=urls, max_history_depth=max_history_depth,
        )

    # -------------------------------------------------------------------------
    # Route Dict
    # -------------------------------------------------------------------------

    routes = {
        "init": init,
        "split": split,
        "merge": merge,
        "undo": undo,
        "reset": reset,
        "ai_split": ai_split,
    }

    return router, routes

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()